In [1]:
# update command line tool -> 
# (1) csv save (with more columns)
# (2) csv to model file update? (new script)
# (3) augment annotation?
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import annotation_maker as am
from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [3]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)

In [4]:
print(recom.getReactionIDs())

['ODC', 'SAMdc', 'SSAT_for_S', 'SSAT_for_D', 'PAO_for_aD', 'PAO_for_aS', 'SpdS', 'SpmS', 'MAT', 'VCoA', 'VacCoA', 'P_efflux', 'aD_efflux']


In [5]:
recom.reactions.exist_annotation

{'ODC': ['RHEA:22964'],
 'SAMdc': ['RHEA:15981'],
 'SSAT_for_S': ['RHEA:33099', 'RHEA:28150', 'RHEA:28270', 'RHEA:11116'],
 'SSAT_for_D': ['RHEA:33099', 'RHEA:28150', 'RHEA:28270', 'RHEA:11116'],
 'PAO_for_aD': ['RHEA:25800', 'RHEA:16133'],
 'PAO_for_aS': ['RHEA:25800', 'RHEA:16133'],
 'SpdS': ['RHEA:12721'],
 'SpmS': ['RHEA:19973'],
 'MAT': ['RHEA:21080']}

In [6]:
recom.recommendReaction(ids=['ODC', 'SSAT_for_S'], min_score=0.0)

                                        ODC (credibility score: 0.817)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  2 | RHEA:22964   |         0.500 | ornithine decarboxylase activity                                      |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  3 | RHEA:59048   |         0.500 | D-ornithine + H(+) = CO2 + putrescine                                 |
+----+----

In [7]:
recom.selectAnnotation(('ODC', [1,2]))

Selection updated.


In [8]:
recom.selectAnnotation(('SSAT_for_S', [1]))

Selection updated.


In [9]:
recom.selection

{'species': {},
 'reaction': {'ODC':                    annotation  match score  \
  ODC (cred. 0.817)                            
  1                  RHEA:28827          1.0   
  2                  RHEA:22964          0.5   
  
                                                                 label  
  ODC (cred. 0.817)                                                     
  1                  L-ornithine(out) + putrescine(in) = L-ornithin...  
  2                                   ornithine decarboxylase activity  ,
  'SSAT_for_S':                           annotation  match score  \
  SSAT_for_S (cred. 0.985)                            
  1                         RHEA:33099          0.8   
  
                                                                        label  
  SSAT_for_S (cred. 0.985)                                                     
  1                         acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...  }}

In [11]:
one_type = 'reaction'
type_selection = recom.selection[one_type]

# if species
TYPE_EXISTING_ATTR = {'species': recom.species.exist_annotation,
                      'reaction': recom.reactions.exist_annotation}


model = recom.sbml_document.getModel()


# re-define 'element_df',
# for k in type_selection.keys():

k = list(type_selection.keys())[1]    
    
element_df = type_selection[k]
fnames = [recom.fname]*element_df.shape[0]
ids = [k]*type_selection[k].shape[0] 
metaids = [model.getReaction(k).meta_id]*type_selection[k].shape[0]
display_names = [model.getReaction(k).name]*type_selection[k].shape[0]
existing = [1 if val in TYPE_EXISTING_ATTR[one_type][ids[idx]] else 0 \
            for idx, val in enumerate(element_df['annotation'])]

element_df.insert(0, "file", fnames)
element_df.insert(1, "id", ids)
element_df.insert(2, "meta id", metaids)
element_df.insert(3, "display name", display_names)
element_df["existing"] = existing

# add additional rows if there is annotation not caught by df
annotation2add = [val for val in TYPE_EXISTING_ATTR[one_type][k] \
                  if val not in list(element_df['annotation'])]
# add new rows

# types = [one_type]*len(ids)
# element_df.insert(1, "type", types)
# element_df["USE ANNOTATION"] = 0

In [12]:
annotation2add

['RHEA:28150', 'RHEA:28270', 'RHEA:11116']

In [42]:
TYPE_EXISTING_ATTR[one_type][k]

['RHEA:33099', 'RHEA:28270', 'RHEA:28150', 'RHEA:11116']

In [16]:
'RHEA:33099' in list(element_df['annotation'])

True

In [13]:
element_df

,file,annotation,id,meta id,display name,match score,label,existing
SSAT_for_S (cred. 0.985),,,,,,,,
1,BIOMD0000000190.xml,RHEA:33099,SSAT_for_S,metaid_0000064,Spermidine/Spermine N1-acetyltransferase for S,0.8,acetyl-CoA + spermine = CoA + H(+) + N(1)-acet...,1


In [15]:
# recom.saveToCSV()

In [14]:
# RES_DIR = os.path.join(AMAS_DIR, 'res')

In [18]:
# CheBI/RHEA match score

one_chebi = 'CHEBI:15351'
one_synonyms = sa.CHEBI_LOW_SYNONYMS[one_chebi]
one_synonyms

['accoa',
 's-acetyl-coa',
 'acetyl-coa',
 'acetyl coenzyme a',
 "3'-phosphoadenosine 5'-(3-{(3r)-4-[(3-{[2-(acetylsulfanyl)ethyl]amino}-3-oxopropyl)amino]-3-hydroxy-2,2-dimethyl-4-oxobutyl} dihydrogen diphosphate)",
 's-acetyl-coenzyme a']

In [24]:
charcount_sub_df

,a,b,c,d,e,f,g,h,i,j,...,0,1,2,3,4,5,6,7,8,9
78498,0.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78499,0.534522,0.000000,0.534522,0.000000,0.267261,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78500,0.554700,0.000000,0.554700,0.000000,0.277350,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78501,0.384900,0.000000,0.384900,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
78502,0.222834,0.037139,0.037139,0.222834,0.259973,0.037139,0.037139,0.297113,0.222834,0.0,...,0.0,0.0,0.111417,0.222834,0.074278,0.037139,0.0,0.0,0.0,0.0
78503,0.377964,0.000000,0.377964,0.000000,0.566947,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [25]:
k

'SSAT_for_S'

In [34]:
chebi_sub_df = sa.CHEBI_DF[sa.CHEBI_DF['chebi']==one_chebi]
charcount_sub_df = sa.CHARCOUNT_DF.loc[chebi_sub_df.index, :]

one_query, name_used = recom.species.prepareCounterQuery(specs=['SAM'],
                                                         ref_cols=charcount_sub_df.columns,
                                                         use_id=True)
multi_mat = charcount_sub_df.dot(one_query)
max_val = np.round(float(multi_mat.max()), cn.ROUND_DIGITS)
max_val

0.799

In [53]:
one_rid = 'SSAT_for_S'
one_rhea = 'RHEA:33099'

specs2predict = recom.reactions.reaction_components[one_rid] 
spec_results = recom.getSpeciesListRecommendation(pred_ids=specs2predict,
                                                  update=False,
                                                  method='cdist')
pred_formulas = dict()
for one_recom in spec_results:
  chebis = [val[0] for val in one_recom.candidates]
  forms = list(set([cn.REF_CHEBI2FORMULA[k] \
         for k in chebis if k in cn.REF_CHEBI2FORMULA.keys()]))
  pred_formulas[one_recom.id] = forms


query_df = pd.DataFrame(0, 
                        index=ra.REF_MAT.columns,
                        columns=[one_rid])
for one_spec_key in pred_formulas.keys():
    one_spec = pred_formulas[one_spec_key]
    # For each one_rid, set the values 1.0
    query_df.loc[[val for val in one_spec if val in query_df.index], one_rid] = 1

sub_ref_mat = ra.REF_MAT.loc[one_rhea, :]
num_matches = int(sub_ref_mat.dot(query_df))
num_cand_elements = len(sub_ref_mat.to_numpy().nonzero()[0])

res_match_score = np.round(num_matches/num_cand_elements, cn.ROUND_DIGITS)


# pred_formulas = {val:recom.species.exist_annotation_formula[val] \
#                    for val in specs2predict \
#                    if val in self.species.exist_annotation_formula.keys()}

# r2pred_spec_formulas[one_rid] = {one_spec:inp_spec_dict[one_spec] \
#                                  for one_spec in recom.reactions.reaction_components[one_rid]}

In [54]:
res_match_score

0.8

In [51]:
num_maxpos_matches

5

In [ ]:
if inp_ids:
  one_query, name_used = recom.species.prepareCounterQuery(specs=inp_ids,
                                                           ref_cols=ref_df.columns,
                                                           use_id=True)
elif inp_strs:
  one_query, name_used = recom.species.prepareCounterQuery(specs=inp_strs,
                                                           ref_cols=ref_df.columns,
                                                           use_id=False)  
else:
  return None
multi_mat = ref_df.dot(one_query)
max_val = multi_mat.max()
result = dict()
for one_spec in one_query.c